In [1]:
# The Usual
import pandas as pd

#NLTK
import nltk; nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import word_tokenize

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# FP-Growth
from mlxtend.frequent_patterns import fpgrowth
from mlxtend.preprocessing import TransactionEncoder

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/tristandealwis/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/tristandealwis/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
listings = pd.read_csv("listings.csv", usecols=['id', 'name', 'description', 'neighbourhood_cleansed', 'zipcode', 'property_type', 'room_type', 
                                            'price', 'availability_365'])
listings = listings.rename(columns={'id': 'listing_id'})

reviews = pd.read_csv("reviews.csv", usecols=['listing_id', 'comments'])

In [3]:
df = pd.merge(listings, reviews, on='listing_id')

In [4]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [5]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(df['comments']))

In [6]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['lisa', 'is', 'wonderful', 'kind', 'and', 'thoughtful', 'host', 'the', 'listing', 'is', 'accurate', 'there', 'is', 'an', 'entire', 'floor', 'for', 'the', 'guest', 'with', 'full', 'bath', 'and', 'kitchen', 'area', 'looking', 'out', 'over', 'leafy', 'backyard', 'everything', 'needed', 'for', 'my', 'one', 'month', 'stay']


In [7]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [8]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)
# print('type(data_words_nostops): ', type(data_words_nostops))

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)
# print('type(data_words_bigrams): ', type(data_words_bigrams))
# print(data_words_bigrams)

# Form Trigrams
data_words_trigrams = make_trigrams(data_words_nostops)

In [9]:
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# ! python3 -m spacy download en

nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

In [10]:
te = TransactionEncoder()
te_ary = te.fit(data_words_trigrams).transform(data_words_trigrams)
df = pd.DataFrame(te_ary, columns=te.columns_)
df

,a_c,aa,aaa,aaaa,aaaaa,aaaaaaa,aaaaaaamazing,aaaaaaand,aaaaand,aaaah,...,ﾛｹｰｼｮﾝ最高,ﾛｹｰｼｮﾝ的には,ﾛﾊﾞｰﾄさんのおかけて,ﾛﾌﾄに遊ひに来るstreet,ﾛﾌﾄのﾍﾞｯﾄもとても大きく,ﾛﾘﾏｰ駅から徒歩,ﾛﾝﾀﾞさん宅に到着した所,ﾜｲｺﾌ,ﾜｲﾝ飲む,ﾜﾝﾙｰﾑなのて
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1166684,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1166685,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1166686,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1166687,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [ ]:
fpgrowth(df, min_support=0.15, use_colnames=True)